<a href="https://colab.research.google.com/github/m-mazurkiewicz/PWR_MachineLearning/blob/AlexNet_master/Comapre_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os
from keras.preprocessing.image import ImageDataGenerator
import keras
from tqdm import tqdm
import skimage.io as io
import numpy as np
import pickle

Using TensorFlow backend.


In [0]:
drive.mount('/content/gdrive/', force_remount=True)

In [0]:
base_dir_models = '/content/gdrive/My Drive/PWr_AlexNet_models/'
base_dir_processed = '/content/gdrive/My Drive/PWr_AlexNet_data/processed/'
number_of_images_for_fit = -1

In [0]:
if os.path.isfile('/content/gdrive/My Drive/PWr_AlexNet_data/models_testing.pickle'):
  models_testing = pickle.load('/content/gdrive/My Drive/PWr_AlexNet_data/models_testing.pickle')
else:
  models_testing = dict()

In [0]:
for i in os.listdir(base_dir_models):
  padding = i
  for j in os.listdir(base_dir_models + i):
    image_preprocess = j
    for k in os.listdir(base_dir_models + i +'/' + j):
      batch_size = k
      for l in os.listdir(base_dir_models + i +'/' + j +'/' + k):
        if 'h5' in l and l[:-3] not in models_testing.keys():
          all_images = []
          for class_name in os.listdir(base_dir_processed + '{0}/{1}'.format(padding,image_preprocess) + '/' + 'train'):
            for image_path in tqdm(os.listdir(base_dir_processed + '{0}/{1}'.format(padding,image_preprocess) + '/' + 'train/' + class_name)[:number_of_images_for_fit]):
              img = io.imread(base_dir_processed + '{0}/{1}'.format(padding,image_preprocess) + '/' + 'train/' + class_name + '/' + image_path)
              all_images.append(img)
          x_train = np.array(all_images)
          del all_images
          image_gen_test = ImageDataGenerator(featurewise_center=True)
          image_gen_test.fit(x_train)
          test_generator = image_gen_test.flow_from_directory(
              base_dir_processed + '{0}/{1}'.format(padding,image_preprocess) + '/' + 'test',
                  target_size=(227,227),
                  batch_size=int(batch_size),
                  class_mode='categorical')
          model = keras.models.load_model(base_dir_models + i +'/' + j +'/' + k + '/' + l)
          metrics = model.evaluate_generator(test_generator, steps=1304//int(batch_size))
          models_testing[l.replace('.h5','')] = metrics[1]
          del x_train
          del model
with open('/content/gdrive/My Drive/PWr_AlexNet_data/models_testing.pickle', 'wb') as f:
  pickle.dump(models_testing, f)